In [1]:
import pandas as pd
import os
import re
import numpy as np

In [2]:
ls data 

0_reviews_Automotive_5_word_vecs.npy
0_reviews_Beauty_5_review_embeds.npy
0_reviews_Beauty_5_word_vecs.npy
0_reviews_Digital_Music_5_word_vecs.npy
0_reviews_Musical_Instruments_5_word_vecs.npy
10_reviews_Beauty_5_review_embeds.npy
10_reviews_Beauty_5_word_vecs.npy
10_reviews_Digital_Music_5_word_vecs.npy
10_reviews_Musical_Instruments_5_word_vecs.npy
11_reviews_Beauty_5_review_embeds.npy
11_reviews_Beauty_5_word_vecs.npy
11_reviews_Digital_Music_5_word_vecs.npy
11_reviews_Musical_Instruments_5_word_vecs.npy
12_reviews_Beauty_5_review_embeds.npy
12_reviews_Beauty_5_word_vecs.npy
12_reviews_Musical_Instruments_5_word_vecs.npy
13_reviews_Beauty_5_review_embeds.npy
13_reviews_Beauty_5_word_vecs.npy
13_reviews_Musical_Instruments_5_word_vecs.npy
14_reviews_Beauty_5_review_embeds.npy
14_reviews_Beauty_5_word_vecs.npy
14_reviews_Musical_Instruments_5_word_vecs.npy
15_reviews_Beauty_5_review_embeds.npy
15_reviews_Beauty_5_word_vecs.npy
15_reviews_Musical_Instruments_5_word_vecs.npy
16_reviews_

In [3]:
cleaned_files = [ f for f in os.listdir("data") if f.endswith(".csv") and f.startswith("reviews_") and "word_vecs" not in f]
cleaned_files

['reviews_Amazon_Instant_Video_5.csv',
 'reviews_Apps_for_Android_5.csv',
 'reviews_Automotive_5.csv',
 'reviews_Baby_5.csv',
 'reviews_Beauty_5.csv',
 'reviews_Cell_Phones_and_Accessories_5.csv',
 'reviews_Clothing_Shoes_and_Jewelry_5.csv',
 'reviews_Digital_Music_5.csv',
 'reviews_Electronics_5_0.csv',
 'reviews_Electronics_5_1.csv',
 'reviews_Electronics_5_2.csv',
 'reviews_Electronics_5_3.csv',
 'reviews_Electronics_5_4.csv',
 'reviews_Electronics_5_5.csv',
 'reviews_Electronics_5_6.csv',
 'reviews_Grocery_and_Gourmet_Food_5.csv',
 'reviews_Health_and_Personal_Care_5.csv',
 'reviews_Home_and_Kitchen_5_0.csv',
 'reviews_Home_and_Kitchen_5_1.csv',
 'reviews_Musical_Instruments_5.csv',
 'reviews_Office_Products_5.csv',
 'reviews_Patio_Lawn_and_Garden_5.csv',
 'reviews_Pet_Supplies_5.csv',
 'reviews_Sports_and_Outdoors_5.csv',
 'reviews_Tools_and_Home_Improvement_5.csv']

In [4]:

def one_hot_label(label):
    if label==0:
        return np.array([1,0])
    else:
        return np.array([0,1])

In [5]:


batchSize = 1024
lstmUnits = 64
numClasses = 2
iterations = 100
maxSeqLength = 250



def review_to_bow_vector(rev):
    rev = re.sub('[^-a-zA-Z0-9_ -]+', '', rev)
    split_words = rev.split()
    
    firstFile = np.zeros((maxSeqLength), dtype='int32')
    indexCounter = 0
    for word in split_words[0:250]:
        try: 
            firstFile[indexCounter] = wordsList.index(word)
        except ValueError:
            firstFile[indexCounter] = 399999 #Vector for unknown words
        indexCounter = indexCounter + 1
    return firstFile


# load the GLOVE arrays, smaller than w2v google, hopefully no perfomance drop
wordsList = np.load('data/wordsList.npy')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('utf-8') for word in wordsList] 
wordVectors = np.load('data/wordVectors.npy')

#good_files = ["data/"+f for f in os.listdir("data") if ".csv" in f]
good_files = ["data/reviews_Musical_Instruments_5.csv", "data/reviews_Automotive_5.csv", "data/reviews_Amazon_Instant_Video_5.csv"]
file_count = -1


In [6]:
frames = []

for f in cleaned_files:
    print(f)
    frame = pd.read_csv("data/"+f, encoding='utf-8')
    print("frame shape", frame.shape, frame.columns)
    
    frame = frame[(frame.clean_text.notnull()) & (frame.clean_text.str.len() > 100)]
    print("frame shape", frame.shape)
    frames.append(frame)
    
data = pd.concat(frames)
print("final shape", data.shape)
    
    

reviews_Amazon_Instant_Video_5.csv
frame shape (32939, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (31427, 2)
reviews_Apps_for_Android_5.csv
frame shape (667816, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (592482, 2)
reviews_Automotive_5.csv
frame shape (19043, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (18299, 2)
reviews_Baby_5.csv
frame shape (143537, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (139679, 2)
reviews_Beauty_5.csv
frame shape (176254, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (169808, 2)
reviews_Cell_Phones_and_Accessories_5.csv
frame shape (173000, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (162479, 2)
reviews_Clothing_Shoes_and_Jewelry_5.csv
frame shape (248252, 2) Index(['clean_text', 'sentiment'], dtype='object')
frame shape (234449, 2)
reviews_Digital_Music_5.csv
frame shape (57917, 2) Index(['clean_text', 'sentiment'], dtype='obje

In [7]:
import sklearn.utils
print(data.iloc[1])

data = sklearn.utils.shuffle(data, random_state=1)
print(data.iloc[1])

clean_text    i highly recommend this series it is a must fo...
sentiment                                                     1
Name: 1, dtype: object
clean_text    these speaker have a small footprint and big s...
sentiment                                                     1
Name: 125914, dtype: object


In [8]:
start_index = 0
split_size = 16384
shuffled_frames = []
while start_index+split_size < data.shape[0]:
    shuffled_frames.append(data[start_index:start_index+split_size])
    start_index += split_size


In [ ]:
i = 6

In [ ]:
file_name = "reviews_shuffled_" + str(i)
label_file_name = "labels_shuffled_" + str(i)
batch = shuffled_frames[i]
print("batch selected",i, batch.shape)
vecs = batch.clean_text.map(review_to_bow_vector)


batch selected 6 (16384, 2)


In [ ]:
arr = np.zeros((split_size, maxSeqLength))
labels = np.zeros((split_size, numClasses))

for j, v in enumerate(vecs):
    arr[j,] = v
    labels[j,] = one_hot_label(batch.sentiment.iloc[j])
    
np.save("data/shuffled/" + file_name, arr)
np.save("data/shuffled/" + label_file_name, labels)
i += 1

In [ ]:
len(vecs)